## DSpy Prompt Optimisation for Multi-Hop (CoT) QA (Baleen Architecture)

A single search query is often not enough for complex QA tasks. For instance, an example within HotPotQA includes a question about the birth city of the writer of "Right Back At It Again". A search query often identifies the author correctly as "Jeremy McKinnon", but lacks the capability to compose the intended answer in determining when he was born.

The standard approach for this challenge in retrieval-augmented NLP literature is to build multi-hop search systems, like GoldEn (Qi et al., 2019) and Baleen (Khattab et al., 2021). These systems read the retrieved results and then generate additional queries to gather additional information when necessary before arriving to a final answer. Using DSPy, we can simulate such systems in a few lines of code.

Refer to https://arxiv.org/pdf/2101.00436.pdf for a explanation of the Baleen architecture.


In [30]:
%env pip install -qU langchain-ibm

env: pip=install -qU langchain-ibm


In [31]:
import dspy
from dsp.utils import deduplicate
from dspy.datasets import HotPotQA
from dspy.predict.retry import Retry
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch
from dspy.evaluate.evaluate import Evaluate
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

import os
import requests
from dsp import LM
import dspy

In [32]:
os.environ['WATSONX_URL']="https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
os.environ['WATSONX_APIKEY']=""
os.environ['WATSONX_PROJECTID']=""

### Create Implementation of DSpy LM Class for watsonx.ai models

In [33]:
class Watson(LM):
    def __init__(self,model,api_key):
        self.kwargs = {
            "model": model,
            "temperature": 0.0,
            "max_tokens": 150,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            "n": 1,
        }
        self.model = model
        self.api_key = api_key
        self.provider = "default"
        self.history = []
        
        self.base_url = os.environ['WATSONX_URL']
        self.project_id = os.environ['WATSONX_PROJECTID']

    def basic_request(self, prompt: str, **kwargs):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Accept": "application/json",
            "content-type": "application/json"
        }

        data = {
            "parameters": {**kwargs},
            "model_id": self.model,
            "input": prompt,
            "project_id": self.project_id
        }

        response = requests.post(self.base_url, headers=headers, json=data)
        response = response.json()

        self.history.append({
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
        })
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.request(prompt, **kwargs)

        print(response)

        completions = [result["generated_text"] for result in response["results"]]

        return completions

### Configure DSpy with the watsonx.ai Language Model

In [34]:
import requests

def generate_access_token(api_key):
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    }
    response = requests.post('https://iam.cloud.ibm.com/identity/token', data=data, headers=headers)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

token = generate_access_token(os.environ['WATSONX_APIKEY'])

In [35]:
watsonx = Watson(model="meta-llama/llama-2-70b-chat",api_key=token)
dspy.settings.configure(lm=watsonx, trace=[], temperature=0.7)

### Load the dataset

We make use of the mentioned HotPotQA dataset, a collection of complex question-answer pairs typically answered in a multi-hop fashion. We can load this dataset provided by DSPy through the HotPotQA class:

In [36]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(rm=colbertv2_wiki17_abstracts)

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

### Build the Signature of the DSpy Modules

We'll start by creating the GenerateAnswer signature that'll take context and question as input and give answer as output.

In [37]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

Unlike usual QA pipelines, we have an intermediate question-generation step in Baleen for which we'll need to define a new Signature for the "hop" behavior: inputting some context and a question to generate a search query to find missing information.

In [38]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

### Create the DSpy Module (Baleen pipeline)

As we can see, the __init__ method defines a few key sub-modules:

- generate_query: For each hop, we will have one dspy.ChainOfThought predictor with the GenerateSearchQuery signature.
- retrieve: This module will conduct the search using the generated queries over our defined ColBERT RM search index via the dspy.Retrieve module.
- generate_answer: This dspy.Predict module will be used with the GenerateAnswer signature to produce the final answer.

The forward method uses these sub-modules in simple control flow.

1. First, we'll loop up to self.max_hops times.
2. In each iteration, we'll generate a search query using the predictor at self.generate_query[hop].
3. We'll retrieve the top-k passages using that query.
4. We'll add the (deduplicated) passages to our context accumulator.
5. After the loop, we'll use self.generate_answer to produce an answer.
6. We'll return a prediction with the retrieved context and predicted answer.

In [39]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

### Executing the Pipeline

In [40]:
# Ask any question you like to this simple RAG program.
my_question = "How many storeys are in the castle that David Gregory inherited?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:38.905Z', 'results': [{'generated_text': ' find the answer. We know that David Gregory inherited a castle from his father, John Gregory. We', 'generated_token_count': 20, 'input_token_count': 114, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:40.170Z', 'results': [{'generated_text': ' How many storeys are in the castle that John Gregory owned?\n\n---\n\nContext:', 'generated_token_count': 20, 'input_token_count': 138, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'messag

In [41]:
watsonx.history

[{'prompt': "Write a simple search query that will help answer a complex question.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the query}. We ...\n\nQuery: ${query}\n\n---\n\nContext: N/A\n\nQuestion: How many storeys are in the castle that David Gregory inherited?\n\nReasoning: Let's think step by step in order to",
  'response': {'model_id': 'meta-llama/llama-2-70b-chat',
   'created_at': '2024-04-08T09:57:38.905Z',
   'results': [{'generated_text': ' find the answer. We know that David Gregory inherited a castle from his father, John Gregory. We',
     'generated_token_count': 20,
     'input_token_count': 114,
     'stop_reason': 'max_tokens'}],
   'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in 

### Compiling DSpy to Optimise the Pipeline

A zero-shot approach quickly falls short for more specialized tasks, novel domains/settings, and more efficient (or open) models.

To address this, DSPy offers compilation. Let's compile our multi-hop (SimplifiedBaleen) program.

Let's first define our validation logic for compilation:

The predicted answer matches the gold answer.
The retrieved context contains the gold answer.
None of the generated queries is rambling (i.e., none exceeds 100 characters in length).
None of the generated queries is roughly repeated (i.e., none is within 0.8 or higher F1 score of earlier queries).

In [42]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False

    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]

    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False

    return True

We'll use one of the most basic teleprompters in DSPy, namely, BootstrapFewShot to optimize the predictors in pipeline with few-shot examples.

In [43]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:49.755Z', 'results': [{'generated_text': ' find the answer. We know that At My Window is a song, so we can start by searching', 'generated_token_count': 20, 'input_token_count': 114, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:51.311Z', 'results': [{'generated_text': ' "At My Window" songwriter\n\n---\n\nContext: N/A\n\nQuestion', 'generated_token_count': 20, 'input_token_count': 138, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Pro

  5%|▌         | 1/20 [00:09<02:51,  9.01s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:57.188Z', 'results': [{'generated_text': ' Townes Van Zandt\n\n---\n\nContext:\n[1] «The song', 'generated_token_count': 20, 'input_token_count': 964, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T09:57:58.625Z', 'results': [{'generated_text': ' find the answer. We know that Candace Kita is an actress who has guest-starred', 'generated_token_count': 20, 'input_token_count': 113, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed 

 10%|█         | 2/20 [00:17<02:41,  8.95s/it]

...

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:00:48.417Z', 'results': [{'generated_text': ' determine who is older. We know that ...\n\nAnswer: Aleksandr Danilovich Aleks', 'generated_token_count': 20, 'input_token_count': 880, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:00:49.798Z', 'results': [{'generated_text': ' find the answer. We know that Diogal Sakho is a Senegalese professional footballer', 'generated_token_count': 20, 'input_token_count': 118, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This mode

 95%|█████████▌| 19/20 [03:08<00:09,  9.41s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:00:57.076Z', 'results': [{'generated_text': ' Atlantic\n\n---\n\nContext:\n[1] «The 1972 FA', 'generated_token_count': 20, 'input_token_count': 1022, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:00:58.543Z', 'results': [{'generated_text': ' find the answer. We know that the guitarist is a member of the Iron Maidens,', 'generated_token_count': 20, 'input_token_count': 129, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a t

100%|██████████| 20/20 [03:18<00:00,  9.93s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:01:06.538Z', 'results': [{'generated_text': ' The composer Johann Strauss II.\n\n---\n\nContext:\n\nQuestion: Which guitar', 'generated_token_count': 20, 'input_token_count': 1152, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
Bootstrapped 2 full traces after 20 examples in round 0.


In [44]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))

    print(gold_titles)

    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))

    print(found_titles)

    return gold_titles.issubset(found_titles)

Define our evaluation function and compare the performance of the uncompiled and compiled Baleen pipelines. While this devset does not serve as a completely reliable benchmark, it is instructive to use for this tutorial.

In [45]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=gold_passages_retrieved, display=False)

compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=gold_passages_retrieved)

print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:01:08.259Z', 'results': [{'generated_text': ' answer this question. First, we need to find the location of Cangzhou. We can', 'generated_token_count': 20, 'input_token_count': 120, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:01:09.786Z', 'results': [{'generated_text': ' Cangzhou location\n\n---\n\nContext: Cangzhou is located in the', 'generated_token_count': 20, 'input_token_count': 144, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Produ

  0%|          | 0/50 [00:00<?, ?it/s]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:08:11.845Z', 'results': [{'generated_text': ' find the answer. We know that Cangzhou is a prefecture-level city in', 'generated_token_count': 20, 'input_token_count': 303, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:08:13.513Z', 'results': [{'generated_text': ' (Cangzhou OR Qionghai) Hebei province\n\n---\n\nContext', 'generated_token_count': 20, 'input_token_count': 327, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a 

Average Metric: 0 / 1  (0.0):   2%|▏         | 1/50 [00:09<08:08,  9.98s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:08:20.085Z', 'results': [{'generated_text': ' Yes\n\n---\n\nContext:\n[1] «The 1972 FA', 'generated_token_count': 20, 'input_token_count': 2429, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'cangzhou', 'qionghai'}
{'cang county', 'xinhua district cangzhou', 'dongguang county', 'cangzhou', 'haixing county'}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:08:21.496Z', 'results': [{'generated_text': ' find the answer. We know that Marc-Andre Fleury was drafted to the Vegas', 'generated_token_count': 20, 'input_token_count': 319, 'stop

...

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:17:30.752Z', 'results': [{'generated_text': ' Clarice Lispector\n\n---\n\nContext:\n[1] «The 1', 'generated_token_count': 20, 'input_token_count': 2490, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'sasha alexander', 'yes man film'}
{'suzana dinić', 'suzanna film', 'sasha alexander', 'hour of star'}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:17:31.881Z', 'results': [{'generated_text': ' find the answer. We know that the Election Law Journal is published by the\n\nQuery:', 'generated_token_count': 20, 'input_token_count': 295,

Average Metric: 21 / 50  (42.0): 100%|██████████| 50/50 [09:30<00:00, 11.42s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:17:41.023Z', 'results': [{'generated_text': ' London', 'generated_token_count': 2, 'input_token_count': 2670, 'stop_reason': 'eos_token'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'election law journal', 'reed college'}
{'dave leips atlas of us presidential elections', 'electoralvotecom', 'john david booth', 'united kingdom election results', 'history of british political parties'}
Average Metric: 21 / 50  (42.0%)


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",Yes --- Context: [1] «The 1972 FA,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","['Marc-Édouard Vlasic | Marc-Édouard Vlasic (born March 30, 1987) is a Canadian professional ice hockey defenceman for the San Jose Sharks of the National Hockey...",The expansion draft. --- Context: [1] «The 19,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[""Æthelflæd | Æthelflæd, Lady of the Mercians ( 870 - 12 June 918) ruled Mercia in the English Midlands from 911 until her death. She...",Alfred the Great,✔️ [True]


## Retrieval Score for uncompiled Baleen: 36.0
## Retrieval Score for compiled Baleen: 42.0
